<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [20]:
# librerias

import pandas as pd
import numpy as np
import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem
import scipy.stats as stats
import pylab as plt

In [2]:
# datos
df = pd.read_csv('data/cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [3]:
df.shape

(90189, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
userid,90189.0,4.998412e+06,2.883286e+06,116.0,2512230.0,4995815.0,7496452.0,9999861.0
sum_gamerounds,90189.0,5.187246e+01,1.950509e+02,0.0,5.0,16.0,51.0,49854.0


In [6]:
#df.info(memory_usage='deep')

Segun el Read Me sum_gamerounds es la sumatoria de los niveles juagados en los 14 primeros días desde la instalación del juego. 
Los columnas de retention_1: si volvió el jugador después del primer día de instalación?
retention_7: si volvió el jugador después del séptimo día de instalación?
De estos datos no me importa cualquier juador que haya jugado menos rondas que el número de niveles donde teóricamente se encuentra la puerta de para o paga

In [7]:
# transformacion
df[df.sum_gamerounds<=30].shape

(57562, 5)

In [8]:
df.groupby('version').mean() 

,userid,sum_gamerounds,retention_1,retention_7
version,,,,
gate_30,4.987564e+06,52.456264,0.448188,0.190201
gate_40,5.009073e+06,51.298776,0.442283,0.182000


In [9]:
retention_1 = df[["userid", "version", "retention_1"]] #Retencion a 1 días
retention_7 = df[["userid", "version", "retention_7"]] #Retención a 7 días

In [10]:
display =retention_1.groupby('version').sum()
display['retention_1']

version
gate_30    20034
gate_40    20119
Name: retention_1, dtype: int64

In [11]:
retention_7.sample()

,userid,version,retention_7
27268,3036972,gate_30,False


# Muestreo

Testeo para **retention_1**:

In [12]:
pd.crosstab(df.version, df.retention_1)

retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


Grupo de control:

In [13]:
retention_1[retention_1.version=='gate_30'].shape #44700 lineas 3 columnas
retention_1.userid[retention_1.version=='gate_30'].value_counts #Comprobando que los usuarios son únicos

<bound method IndexOpsMixin.value_counts of 0            116
1            337
6           1066
11          2101
13          2179
          ...   
90179    9998576
90180    9998623
90182    9999178
90183    9999349
90186    9999710
Name: userid, Length: 44700, dtype: int64>

In [14]:
retention_1[retention_1.version=='gate_40'].shape #45.489 líneas, 3 columnas
retention_1.userid[retention_1.version=='gate_40'].value_counts #45.489 líneas, todos los users son únicos

<bound method IndexOpsMixin.value_counts of 2            377
3            483
4            488
5            540
7           1444
          ...   
90181    9998733
90184    9999441
90185    9999479
90187    9999768
90188    9999861
Name: userid, Length: 45489, dtype: int64>

In [2]:
conversion=retention_1.groupby('version')['retention_1']

std=lambda x: np.std(x, ddof=0)             # std poblacional
error_std=lambda x: stats.sem(x, ddof=0)    # error estandard (std/sqrt(n))

conversion=conversion.agg([np.mean, std, error_std])

conversion.columns=['conv%', 'std', 'error_std']

conversion.style.format('{:.4f}') #le damos 4 decimales al formato de salida

NameError: name 'retention_1' is not defined

In [3]:
plt.figure(figsize=(8,6))

sns.barplot(x=retention_1['version'], 
            y=retention_1['retention_1'], 
            ci=95
           )

plt.ylim(0, 0.6)
plt.title('Tasa de conversión por grupo')
plt.xlabel('Grupo')
plt.ylabel('% Conversión');

NameError: name 'plt' is not defined

In [4]:
control_res=retention_1[retention_1.version=='gate_30']['retention_1']
tratamiento_res=retention_1[retention_1.version=='gate_40']['retention_1']

n_con=control_res.count()
n_trat=tratamiento_res.count()

conversiones=[control_res.sum(), tratamiento_res.sum()]  # conversiones por grupo

observaciones=[n_con, n_trat] # cuenta N de cada grupo

z_score, p_value = proportions_ztest(conversiones, nobs=observaciones)

z_score, p_value

NameError: name 'retention_1' is not defined

In [ ]:
(gate_30_a, gate_40_a), (gate_30_b, gate_40_b) = proportion_confint(conversiones,nobs=observaciones,alpha=0.05)

print('Gate 30', [gate_30_a, gate_30_b])
print('Gate 40', [gate_40_a, gate_40_b])

In [16]:
# testeo para retention_7



In [17]:
# conclusiones
# Explica detalladamente las conclusiones obtenidas de. tu testeo.